## Use plain attributes instead of get and set methods
Programmers coming from other languages naturally try to implement getter and setter methods.

In [1]:
class OldResister(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms
        
resister = OldResister(1000)
resister.get_ohms()

1000

In [2]:
resister.set_ohms(1500)
resister.get_ohms()

1500

You should always start implementations with simple public attributes

In [3]:
class Resister(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resister(50e3)
r1.ohms

50000.0

Later, if you decide that you need special behavior when an attribute is set, you can migrate to the @property decorator:

In [4]:
class VoltageResistance(Resister):
    def __init__(self, ohms):
        super(VoltageResistance, self).__init__(ohms)
        self.voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
    def __str__(self):
        return "Voltage = %d, Current = %0.2f" % (self.voltage, self.current)
        
r2 = VoltageResistance(1e3)
print(r2)

Voltage = 0, Current = 0.00


In [5]:
r2.voltage = 10
print(r2)

Voltage = 10, Current = 0.01


Specifying a setter also lets you perform type checking and validation on values passed to your class.

## Consider @property instead of refactoring attributes
One advanced but common use of @property is transitioning what was once a simple numerical attribute into an on the fly calculation.

In [6]:
class Test(object):

    def __init__(self, a, b):
        self.a = a
        self.b = b

    @property
    def c(self):
        return self.a + self.b
    
test = Test(2, 3)
test.c

5

## Use descriptors for reuseable @property methods
The descriptor protocol defines how attribute access is interpreted by the language. Descriptors are objects with any of __get__, __set__, __delete__.

In [7]:
# from the docs at https://docs.python.org/2/howto/descriptor.html
class RevealAccess(object):
    """A data descriptor that sets and returns values
       normally and prints a message logging their access.
    """

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print 'Retrieving', self.name
        return self.val

    def __set__(self, obj, val):
        print 'Updating', self.name
        self.val = val
        
class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
    

In [8]:
m = MyClass()
m.x

Retrieving var "x"


10

In [9]:
m.x = 20

Updating var "x"


In [10]:
m.x

Retrieving var "x"


20

Going through a quick example of when objects are destroyed.

In [11]:
class Foo(object):
    def __init__(self):
        self.obj = None
        print 'created'

    def __del__(self):
        print 'destroyed'

    def show(self):
        print self.obj

    def store(self, obj):
        self.obj = obj

In [12]:
a = Foo()

created


In [13]:
b = a

In [14]:
del a

In [15]:
del b

destroyed


There were two references to the foo object, so the foo object is not destroyed / garbage collected until both a and b are deleted. Weak references, on the other hand, have no effect on the reference count for an object.

In [16]:
import weakref
a = Foo()

created


In [17]:
b = weakref.ref(a)

In [18]:
b

<weakref at 00000000066E2C28; to 'Foo' at 00000000066DF208>

In [19]:
print(b()) # call the weak reference and () to get a strong reference to the object

In [20]:
del a # if we delete the one and only strong reference to our object it is immediately destroyed.

destroyed


In [21]:
b

<weakref at 00000000066E2C28; dead>

In [22]:
print(b())

None


A quick tour of WeakKeyDictionary and WeakValueDictionary

In [23]:
class Foo(object):
    pass

f = Foo()
d = {} 
d["f"] = f 

print dict(d)
del f 
print dict(d) # strong reference to f: instance of foo remains in the dict after f is deleted

{'f': <__main__.Foo object at 0x00000000066DF5F8>}
{'f': <__main__.Foo object at 0x00000000066DF5F8>}


In [24]:
from weakref import WeakValueDictionary

f = Foo()
d = WeakValueDictionary()
d["f"] = f 

print dict(d)
del f 
print dict(d) # weak reference to f: instance of foo is deleted from the dict after f is deleted

{'f': <__main__.Foo object at 0x00000000066DF748>}
{}


In [25]:
from weakref import WeakKeyDictionary
# The WeakKeyDictionary works similarly but uses weak references for the keys instead of the values in the dictionary.

f = Foo()
d = WeakKeyDictionary()
d[f] = 7 

print dict(d)
del f 
print dict(d)

{<__main__.Foo object at 0x00000000066DF048>: 7}
{}


When our underlyting object is deleted it'll automatically get discarded from the WeakKeyDictionary. This assists in ensuring that there is not a memory leak where the dict gets larger and larger without garbage collection when objects are no longer needed.

In [26]:
f = Foo()
g = Foo()
d = {f: "fname", g: "gname"}
# The method get() returns a value for the given key. If key is not available then returns default value None.
print(d.get(f, 0))
print(d[f])

fname
fname


Returning to the main book example. Reuse the behavior and validation of @property methods by defining your own
descriptor classes:

In [27]:
import weakref

class Grade(object):
    '''
      The instance is kept in the key of the dict
      The value (being grade in this case) is kept in the value portion of the dict
      Validation can be placed in the __set__ method
      If the underlying object is deleted then the key value pair for this instance
      is discarded from the WeakKeyDict (which prevents memory leaks)
    '''
    def __init__(self):
        self._values = weakref.WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value


In [28]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
first_exam = Exam()
first_exam.writing_grade = 82

second_exam = Exam()
second_exam.writing_grade = 75

print('First ', first_exam.writing_grade, 'is right')
print('Second', second_exam.writing_grade, 'is right')


('First ', 82, 'is right')
('Second', 75, 'is right')


## Use __getattr__, __getattribute__, and __setattr__ for Lazy Attributes
If your class defines __getattr__, that method is called every time an attribute can’t be found in an object’s instance dictionary.

In [29]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value
    
data = LazyDB()
print('Before:', data.__dict__)
print('foo: ', data.foo)
print('After: ', data.__dict__)

('Before:', {'exists': 5})
('foo: ', 'Value for foo')
('After: ', {'foo': 'Value for foo', 'exists': 5})


Understand that __getattr__ only gets called once when accessing a missing attribute, whereas __getattribute__ gets called every time an attribute is accessed.

In [39]:
class AnotherLazyDB(object):
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super(AnotherLazyDB, self).__getattribute__(name) # look in the base class
        except AttributeError:
            value = 'Value for %s' % name
        setattr(self, name, value)
        return value
    
data = AnotherLazyDB()
print(data.exists)
print(data.__dict__)

Called __getattribute__(exists)
5
Called __getattribute__(__dict__)
{'exists': 5}


In [40]:
print(data.one)
print(data.__dict__)

Called __getattribute__(one)
Value for one
Called __getattribute__(__dict__)
{'exists': 5, 'one': 'Value for one'}


In [41]:
print(data.one)
print(data.__dict__)

Called __getattribute__(one)
Value for one
Called __getattribute__(__dict__)
{'exists': 5, 'one': 'Value for one'}


Avoid infinite recursion in __getattribute__ and __setattr__ by using methods from super() (i.e., the object class) to access instance attributes directly. Why do lazy evaluation? One example of this is instance attributes that take long to compute. With lazy evaluation we calculate only when needed and then store the result for any future reference.